# Notebook to process the ERA5 wind data for regression



In [26]:
import xarray as xr
import numpy as np

In [27]:
path = '/projects/0/einf2878/ROMS/data/ERA/raw/6hourly'

data = xr.open_mfdataset(f'{path}/*10.nc')

In [20]:
# Obtain annual averages
data_annual = data.groupby('time.year').mean('time').rename({'year' : 'time'})

In [21]:
data_annual

<xarray.Dataset>
Dimensions:    (longitude: 137, latitude: 121, time: 75)
Coordinates:
  * longitude  (longitude) float32 -22.0 -21.75 -21.5 -21.25 ... 11.5 11.75 12.0
  * latitude   (latitude) float32 64.0 63.75 63.5 63.25 ... 34.5 34.25 34.0
  * time       (time) int64 1948 1949 1950 1951 1952 ... 2019 2020 2021 2022
Data variables:
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(1, 121, 137), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(1, 121, 137), meta=np.ndarray>

In [22]:
# Obtain wind stress
wind_stress = data_annual.assign(u2 = data_annual.u10*(np.sqrt(data_annual.u10**2+data_annual.v10**2)), v2 = data_annual.v10*(np.sqrt(data_annual.u10**2+data_annual.v10**2)))

In [23]:
wind_stress

<xarray.Dataset>
Dimensions:    (longitude: 137, latitude: 121, time: 75)
Coordinates:
  * longitude  (longitude) float32 -22.0 -21.75 -21.5 -21.25 ... 11.5 11.75 12.0
  * latitude   (latitude) float32 64.0 63.75 63.5 63.25 ... 34.5 34.25 34.0
  * time       (time) int64 1948 1949 1950 1951 1952 ... 2019 2020 2021 2022
Data variables:
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(1, 121, 137), meta=np.ndarray>
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(1, 121, 137), meta=np.ndarray>
    u2         (time, latitude, longitude) float32 dask.array<chunksize=(1, 121, 137), meta=np.ndarray>
    v2         (time, latitude, longitude) float32 dask.array<chunksize=(1, 121, 137), meta=np.ndarray>

In [24]:
# Drop wind speed
wind_stress = wind_stress.drop(['u10', 'v10'])

In [25]:
wind_stress.to_netcdf('/projects/0/einf2878/ROMS/data/ERA/wind_stress/wind_stress.nc')